In [1]:
import numpy as np

In [2]:
text = "The dog chased the cat"

In [3]:
dataPipes1 = np.zeros(shape=[5, 6])
dataPipes1

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

# self attention layer

## introduction

#### IF X = token embeddings / hidden states ของคำทั้งหมด

In [4]:
TheWord = np.array([1, 0, 1, 0, 0, 0])
dogWord = np.array([0, 0, 1, 0, 0, 0])
chasedWord = np.array([0, 1, 0, 0, 0, 0])
theWord = np.array([0, 1, 1, 0, 0, 0])
catWord = np.array([1, 0, 0, 0, 0, 0])

In [5]:
X = np.array([TheWord, dogWord, chasedWord, theWord, catWord])
print(X)

[[1 0 1 0 0 0]
 [0 0 1 0 0 0]
 [0 1 0 0 0 0]
 [0 1 1 0 0 0]
 [1 0 0 0 0 0]]


In [6]:
dataPipes1 = X

#### K Q V ถูกสร้างจาก X ดังนี้
* $Q = X \cdot W_q$ - Q (query) : ฉันกำลังมองหาอะไร
* $K = X \cdot W_k$ - K (Key) : ฉันมีอะไรให้คนอื่นสนใจ
* $V = X \cdot W_v$ - V (value) : ข้อมูลจริงที่จะส่งต่อ

X คือคำ  
Q คือคำถาม  
K คือป้ายชื่อ  
V คือเนื้อหา

* $X ∈ R^{(n × d_{model})}$  
* $n$ = จำนวน tokens    
* $d_{model}$ = embedding size  
* $W_q ∈ R^{(d_{model} × d_k)}$  
* $W_k ∈ R^{(d_{model} × d_k)}$  
* $W_v ∈ R^{(d_{model} × d_v)}$  

In [7]:
n = 5 # 5 คำ ให้เป็น 5 โทเคน
d_model = 6 # example
num_heads = 2 # จำนวน attentions mattrix ที่ทำการคำนวณขนานกัน

In [8]:
#d_k = d_v = d_model / num_heads
d_k = int(d_model / num_heads)
d_v = int(d_model / num_heads)

In [9]:
W_Q1 = np.random.rand(d_model, d_k)
W_K1 = np.random.rand(d_model, d_k)
W_V1 = np.random.rand(d_model, d_v)

In [10]:
W_Q1

array([[0.81830126, 0.4798185 , 0.01538334],
       [0.69398993, 0.82199585, 0.05002724],
       [0.49895333, 0.0422094 , 0.32964272],
       [0.73396407, 0.50923967, 0.85549981],
       [0.93505267, 0.44647419, 0.5043404 ],
       [0.75852473, 0.15351518, 0.88019687]])

In [11]:
W_Q2 = np.random.rand(d_model, d_k)
W_K2 = np.random.rand(d_model, d_k)
W_V2 = np.random.rand(d_model, d_v)

In [12]:
W_Q2

array([[0.27652169, 0.024888  , 0.56727988],
       [0.51801432, 0.20822402, 0.96009243],
       [0.66138474, 0.20573959, 0.65874378],
       [0.99802171, 0.64181074, 0.38055113],
       [0.65929664, 0.53076932, 0.17946725],
       [0.45873595, 0.68923804, 0.8505974 ]])

### attention #1

In [13]:
Q1 = np.dot(X, W_Q1)
K1 = np.dot(X, W_K1)
V1 = np.dot(X, W_V1)
Q1

array([[1.31725459, 0.5220279 , 0.34502606],
       [0.49895333, 0.0422094 , 0.32964272],
       [0.69398993, 0.82199585, 0.05002724],
       [1.19294326, 0.86420525, 0.37966996],
       [0.81830126, 0.4798185 , 0.01538334]])

### attention #2

In [14]:
Q2 = np.dot(X, W_Q2)
K2 = np.dot(X, W_K2)
V2 = np.dot(X, W_V2)
Q2

array([[0.93790643, 0.23062759, 1.22602366],
       [0.66138474, 0.20573959, 0.65874378],
       [0.51801432, 0.20822402, 0.96009243],
       [1.17939906, 0.41396361, 1.61883621],
       [0.27652169, 0.024888  , 0.56727988]])

### softmax
    

In [15]:
def softmax(x):
    x = x - np.max(x) # stability ป้องกันตัวเลข Overflow เป็นมาตรฐานใน deeplearning เพื่อป้องกันไม่ให้ค่าเฟ้อจากฟังก์ชัน exponential
    # print("current x is : ")
    # print(x)
    exp_x = np.exp(x)
    # print("current exp_x is : ")
    # print(exp_x)
    return exp_x / np.sum(exp_x)

## Attention Head 1

#### attention score of head 1

In [16]:
print(Q1.shape)
print(K1.shape)

(5, 3)
(5, 3)


$$score = \frac{Q \cdot K^T}{d_K} $$

In [17]:
scores1 = np.dot(Q1, K1.T) / np.sqrt(d_k)
scores1

array([[1.13810738, 0.31580162, 0.3982908 , 0.71409242, 0.82230576],
       [0.45526223, 0.11150973, 0.15922779, 0.27073752, 0.34375251],
       [0.73271185, 0.31682229, 0.37318217, 0.69000446, 0.41588956],
       [1.18797408, 0.42833202, 0.53240996, 0.96074198, 0.75964207],
       [0.68284515, 0.20429189, 0.23906301, 0.4433549 , 0.47855326]])

$$scores ∈ R^{(n_{token} \times n_{token})}$$

#### apply softmax row by row 

In [18]:
attn1 = np.apply_along_axis(softmax, axis=1, arr=scores1)

In [19]:
attn1

array([[0.30300855, 0.1331472 , 0.14459611, 0.19829306, 0.22095508],
       [0.23930038, 0.16968896, 0.17798249, 0.19897812, 0.21405005],
       [0.24725488, 0.16312718, 0.17258503, 0.23691758, 0.18011534],
       [0.29109422, 0.13618373, 0.15112131, 0.2319252 , 0.18967555],
       [0.25883013, 0.16039167, 0.16606676, 0.20370681, 0.21100464]])

In [20]:
output1 = np.dot(attn1, V1)
output1

array([[0.8577817 , 0.45335908, 1.19420813],
       [0.85380883, 0.42924615, 1.14458818],
       [0.90917255, 0.43023817, 1.18655776],
       [0.90478968, 0.44801159, 1.21592661],
       [0.86273674, 0.43538312, 1.16500048]])

##### อธิบายความหมายของค่าที่ได้

In [21]:

#โฟกัสที่คำว่า "chased"

word_index = 2 # chased
print("attention weight: ", attn1[word_index])

attention weight:  [0.24725488 0.16312718 0.17258503 0.23691758 0.18011534]


ตรงนี้มี่ความหมายว่าคำว่า "chased" มีความสนใจคำอื่นมากแค่ไหน (อาจหมายถึงคำว่า "chased" มีความสัมพันธ์กับคำอื่นแค่ไหน)

In [22]:
print("output vector: ", output1[word_index])

output vector:  [0.90917255 0.43023817 1.18655776]


เป็นเวคเตอร์ใหม่ของคำว่า "chased" ที่มีบริบทรวมอยู่เรียบร้อยแล้ว

## Attention head 2

In [23]:
scores2 = np.dot(Q2, K2.T)/np.sqrt(d_k)
print("scores from second attendtion head 2 ->")
print(scores2)

attn2 = np.apply_along_axis(softmax, axis=1, arr=scores2);
print("attention2 =>")
print(attn2)

output2 = np.dot(attn2, V2)
print("output 2 =>");
print(output2)

scores from second attendtion head 2 ->
[[0.98442327 0.31583861 1.28220996 1.59804856 0.66858467]
 [0.68149596 0.23606163 0.80719651 1.04325814 0.44543433]
 [0.64310109 0.22939446 0.89550793 1.1249024  0.41370663]
 [1.32459705 0.46545609 1.70270444 2.16816053 0.85914096]
 [0.30292731 0.07977698 0.47501345 0.55479042 0.22315034]]
attention2 =>
[[0.18397231 0.09427354 0.24778761 0.33981813 0.1341484 ]
 [0.19999688 0.1281072  0.22678497 0.28716701 0.15794394]
 [0.18648654 0.12330395 0.24003047 0.30191948 0.14825957]
 [0.17136404 0.07257708 0.25010918 0.39835901 0.10759068]
 [0.19239957 0.15391861 0.22852838 0.24750664 0.1776468 ]]
output 2 =>
[[0.89207677 0.39559131 0.92594229]
 [0.86508664 0.39282388 0.94441967]
 [0.86778963 0.38933798 0.92610926]
 [0.91739922 0.39605922 0.92209605]
 [0.83802694 0.38565578 0.93015361]]


## Now we got the 2 output from both attention head 1 and 2

#### let concat it together

In [24]:
output = np.concatenate([output1, output2], axis=1)
output

array([[0.8577817 , 0.45335908, 1.19420813, 0.89207677, 0.39559131,
        0.92594229],
       [0.85380883, 0.42924615, 1.14458818, 0.86508664, 0.39282388,
        0.94441967],
       [0.90917255, 0.43023817, 1.18655776, 0.86778963, 0.38933798,
        0.92610926],
       [0.90478968, 0.44801159, 1.21592661, 0.91739922, 0.39605922,
        0.92209605],
       [0.86273674, 0.43538312, 1.16500048, 0.83802694, 0.38565578,
        0.93015361]])

In [25]:
output.shape

(5, 6)

In [26]:
X.shape

(5, 6)

now output from attention layer have the same dimension and size as X

## Output projection after concat output

**Output Projection** คือ **Linear transformation** สุดท้ายหลังจากรวม (concat) ผลลัพธ์ของทุก attention heads 
เพื่อผสมมุมมองทั้งหมด และทำให้ขนาดกลับไปเป็น $d_{model}$

สัญลักษณ์ใน Paper คือ $W_O$

ดำเนินการหลังจาก concat เป็น output จาก multi-head attention เรียบร้อยแล้วโดยการ

$$Output_{final} = Concat(output_1, output_2, ... , output_n) \cdot W_O$$

* ส่วนนี้เกิดขึ้นมาจากการที่แต่ละ Head ทำงานแยกขาดจากกัน
* ทำให้ไม่มีการผสมข้อมูลระหว่างแต่ละ attention head 
* ทำให้โมเดลเสียความสามารถในการเรียนรู้

**Output Projection** ทำหน้าที่
* ผสมข้อมูลจากทุก Head 
* เรียนรู้ว่าควรเน้นข้อมูลจาก Head ไหน
* จัด alignment กลับเข้าสู่ space หลัก ($d_{model}$)

In [27]:
#example W_O

W_O = np.random.rand(d_model, d_model)
W_O

array([[0.08976383, 0.88118344, 0.1924429 , 0.64805398, 0.47990694,
        0.67545282],
       [0.74720993, 0.30277765, 0.89273845, 0.5253904 , 0.303565  ,
        0.04119523],
       [0.84168297, 0.20510432, 0.21852152, 0.6910616 , 0.49614686,
        0.38554879],
       [0.23865477, 0.12949836, 0.79392768, 0.09627037, 0.86163337,
        0.31830623],
       [0.55183311, 0.4297852 , 0.77969525, 0.18491824, 0.74328665,
        0.48942342],
       [0.45067858, 0.88596152, 0.82995309, 0.90588485, 0.08091567,
        0.36730597]])

In [28]:
attention_output = np.dot(output, W_O)
attention_output

array([[2.26939794, 2.24395828, 2.61593903, 2.61718043, 2.27938606,
        1.87616218],
       [2.20961851, 2.23466501, 2.57455432, 2.58127494, 2.22172335,
        1.85019572],
       [2.24112384, 2.27498863, 2.5794968 , 2.62970687, 2.26767337,
        1.89624227],
       [2.29247005, 2.28828908, 2.64223425, 2.6588835 , 2.332953  ,
        1.92294364],
       [2.21534327, 2.2293528 , 2.54729911, 2.58753719, 2.20820054,
        1.84698733]])

In [29]:
dataPipes1 = attention_output

**from paper :**  
Output projection lets the model recombine information from different attention heads into a unified representation.

## Add & Norm step after get final output from multi-head attention

ใน 1 Transformer block จะมีการ Add & Norm 2 ครั้ง

ครั้งแรกเกิดหลังจากการทำ Multi-head Attention
ครั้งที่สองเกิดหลังการทำ Feed-Forward Network

โดยการนำ Embedding Vector หรือ X มาบวกเข้ากับ Final output จาก multi-head attention แล้วทำการ Normalized เพื่อให้ค่าใน Vector ไม่โตจนเกินไป

### การ ADD

$$Y = X + F(X)$$
โดย 
* $X$ คือ input เดิม
* $F(X)$ คือ output จาก multi-head attention or FFN

กระบวนการนี้เกิดจากการที่
* เมื่อค่าที่ได้จาก transformer block ผ่านไปยังอีก block ที่อยู่ลึกมาก ๆ แล้ว gradient หายไประหว่างทาง (Vanishing gradient)
* ทำให้การ training ทำได้ยาก
* และข้อมูลเดิมหายระหว่างทาง

โมเดลควรเรียนรู้ว่า "ฉันควรเพิ่มอะไรจากของเดิม"

In [30]:
residual1 = X + dataPipes1
residual1

array([[3.26939794, 2.24395828, 3.61593903, 2.61718043, 2.27938606,
        1.87616218],
       [2.20961851, 2.23466501, 3.57455432, 2.58127494, 2.22172335,
        1.85019572],
       [2.24112384, 3.27498863, 2.5794968 , 2.62970687, 2.26767337,
        1.89624227],
       [2.29247005, 3.28828908, 3.64223425, 2.6588835 , 2.332953  ,
        1.92294364],
       [3.21534327, 2.2293528 , 2.54729911, 2.58753719, 2.20820054,
        1.84698733]])

In [31]:
X

array([[1, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 0]])

In [32]:
residual1

array([[3.26939794, 2.24395828, 3.61593903, 2.61718043, 2.27938606,
        1.87616218],
       [2.20961851, 2.23466501, 3.57455432, 2.58127494, 2.22172335,
        1.85019572],
       [2.24112384, 3.27498863, 2.5794968 , 2.62970687, 2.26767337,
        1.89624227],
       [2.29247005, 3.28828908, 3.64223425, 2.6588835 , 2.332953  ,
        1.92294364],
       [3.21534327, 2.2293528 , 2.54729911, 2.58753719, 2.20820054,
        1.84698733]])

### Norm (Layer Normalization) 


ทำให้ค่าใน vector 
* $mean \approx 0$
* $variance \approx 1$

$$LayerNorm(x) = \frac{x - \mu}{\sigma} \cdot \gamma + \beta$$

ในกรณีที่ไม่มีการ Normalized 
* ทำให้ Vector โตขึ้นเรื่อย ๆ
* scale เพี้ยนเรื่อย ๆ 
* training ไม่นิ่ง

In [33]:
def layer_norm (x, eps=1e-6):
    mean = np.mean(x, axis=1, keepdims=True)
    print("mean = ", mean)
    std = np.std(x, axis=1, keepdims=True)
    print("std = ", std)
    return (x - mean) / (std + eps)

In [34]:
norm1 = layer_norm(residual1)

mean =  [[2.65033732]
 [2.44533864]
 [2.48153863]
 [2.68962892]
 [2.43912004]]
std =  [[0.6081143 ]
 [0.54738058]
 [0.4295777 ]
 [0.59715372]
 [0.42508825]]


In [35]:
norm1

array([[ 1.01799876, -0.66825985,  1.58785959, -0.05452402, -0.61000152,
        -1.27307296],
       [-0.43063219, -0.38487527,  2.0629406 ,  0.24833919, -0.40851812,
        -1.0872542 ],
       [-0.55965249,  1.84704225,  0.22803312,  0.34491524, -0.49784885,
        -1.36248926],
       [-0.66508538,  1.00252103,  1.5952404 , -0.05148653, -0.59729231,
        -1.28389721],
       [ 1.82602414, -0.49346635,  0.25448556,  0.34914349, -0.54322593,
        -1.39296091]])

### eps value

eps = $\epsilon$ 
* เป็นค่าคงที่เล็ก ๆ ใช้ป้องกันการหารด้วย 0
* มักจะมีค่าเท่ากับ $1 \times 10^{-6}$ เนื่องจากเป็นค่าที่เล็กพอที่จะไม่รบกวนผลลัพธ์แต่ใหญ่พอที่จะป้องกัน numerical error

### $\beta$ and $\gamma$ หายไปไหน

$\beta$ และ $\gamma$
เป็นค่าที่ได้จากการ training 
ทำให้โมเดลสามารถเลือกได้ว่า 
* dimension ไหนควรเพิ่ม - ลด และ 
* dimension ไหนควรเลื่อนค่า

In [36]:
#for example beta and gamma after training method is 
beta1 = np.array([2, 0.5, 2, 1, 1, 1])
gamma1 = np.array([0.1, -0.2, 0.3, -0.12, 0.32, 0.2])

In [37]:
y = gamma1 * norm1 + beta1
y

array([[2.10179988, 0.63365197, 2.47635788, 1.00654288, 0.80479951,
        0.74538541],
       [1.95693678, 0.57697505, 2.61888218, 0.9701993 , 0.8692742 ,
        0.78254916],
       [1.94403475, 0.13059155, 2.06840994, 0.95861017, 0.84068837,
        0.72750215],
       [1.93349146, 0.29949579, 2.47857212, 1.00617838, 0.80886646,
        0.74322056],
       [2.18260241, 0.59869327, 2.07634567, 0.95810278, 0.8261677 ,
        0.72140782]])

In [38]:
dataPipes1 = norm1

# Feed Forward neural network

สิ่งที่ได้จากกระบวนการนี้
$$H \in R^{(n \times d_{model})}$$

* FFN ทำงานกับ H ทีละ token (row-wise)
* FFN จะไม่ดู token อื่นเลย (ไม่ขึ้นกับ token อื่น)

FFN definition from Attention is All you need: 
    $$FFN(x) = max(0, x \cdot W_1 + b_1)W_2 + b_2$$

(ของใหม่จะใช้ GELU แทน ReLU)

## 1st linear (expand)

expand dimension from 6 to 24 (4 times of $n_{model}$)

In [39]:
# this value came from training method
W_FFN_Expand_1 = np.random.rand(6, 24) 
W_FFN_Shrink_1 = np.random.rand(24, 6)

In [40]:
# this value came from training method
b_FFN_Expand_1 = np.random.rand(1, 24)
b_FFN_Shrink_2 = np.random.rand(1, 6)

#### *add by using broadCasting*
 * $dataPipes_1 \in R^{(n_{model} \times 5)}$ : result from Plus & Norm process have same dimensions of X 
 * $W_1 \in R^{(n_{model} \times 4n_{model})}$ : layer แรกเป็นเลเยอร์ขยายผลลัพธ์ที่ได้จากเลเยอร์นี้มักจะมีขนาดมิติเป็น 4 เท่าของค่า $n_{model}$

In [41]:
h1 = np.dot(dataPipes1, W_FFN_Expand_1) + b_FFN_Expand_1
h1.shape

(5, 24)

### pass to Relu function

In [42]:
h_prime = np.maximum(0, h1) # if child of h1 less than 0 return 0 instead of minus value inside h1
h_prime

array([[0.        , 0.64130923, 0.        , 0.47872405, 0.        ,
        1.01006683, 1.2041562 , 0.70676377, 0.10125673, 1.34658436,
        1.21382356, 0.        , 0.13963233, 0.39420191, 0.52604075,
        0.66700481, 0.44141633, 0.        , 0.54018419, 0.61516564,
        0.10487459, 0.06964364, 1.19117246, 2.06861511],
       [0.36180272, 1.1184323 , 0.83193107, 0.75214634, 0.        ,
        0.7749543 , 1.66307686, 1.37957596, 0.34334037, 1.16944736,
        0.771336  , 0.        , 0.        , 0.33386591, 1.27551776,
        1.22735931, 1.26937401, 0.15708161, 1.28360083, 0.73188105,
        0.        , 0.        , 1.23752023, 1.48332553],
       [0.32491618, 0.        , 1.14604226, 0.        , 0.48858003,
        0.45343959, 1.87157306, 1.84019436, 0.44854943, 1.08053841,
        0.        , 0.28921823, 1.13133988, 0.83444737, 0.51896314,
        0.03113832, 0.88464613, 0.28425659, 2.45432288, 0.5496055 ,
        0.98823541, 0.38888666, 1.69089595, 1.88321393],
       [0.757

## 2st linear (Shrink)

then dot with shrink weight and shrink bias again

In [43]:
output = np.dot(h_prime, W_FFN_Shrink_1) + b_FFN_Shrink_2
output.shape

(5, 6)

In [44]:
print(output)

[[ 7.65656445  7.89325024  6.71185691  6.91058283  7.66534591  6.05377159]
 [10.56076316 10.26097141  9.55592168  9.01464577  9.58599893  9.55352737]
 [10.15782726 11.7342027   9.83024226 10.32595184 10.40187043 10.0575946 ]
 [11.82377051 11.839737   10.3053274  10.36803637 10.88230038 11.02587045]
 [ 5.32173316  6.8732731   5.33360664  5.4232815   6.59783297  4.32747151]]


## conclude

In [45]:
def ffn(X, W1, b1, W2, b2):
    h = np.dot(X, W1) + b1
    h = np.maximum(0, h)
    h = np.dot(h, W2) + b2
    return h

In [46]:
dataPipes1 = ffn(dataPipes1, W_FFN_Expand_1, b_FFN_Expand_1, W_FFN_Shrink_1, b_FFN_Shrink_2)
dataPipes1.shape

(5, 6)

# Add and Norm once again at the end of transformer block

In [ ]:
addResult = dataPipes1 + norm1
norm2 = layer_norm(addResult)
norm2.shape

mean =  [[ 7.14856199]
 [ 9.75530472]
 [10.41794818]
 [11.04084035]
 [ 5.64619981]]
std =  [[1.24904645]
 [0.98875288]
 [1.53319467]
 [1.06309762]
 [1.31226383]]


(5, 6)

In [48]:
dataPipes1 = norm2
dataPipes1

array([[ 1.22173199,  0.06118935,  0.92162592, -0.234181  , -0.07463095,
        -1.89573531],
       [ 0.37908953,  0.12216531,  1.88475371, -0.49791942, -0.58439609,
        -1.30369304],
       [-0.53468284,  2.06320487, -0.23459028,  0.16496192, -0.33519962,
        -1.12369404],
       [ 0.11085028,  1.69449724,  0.80869962, -0.68130134, -0.710971  ,
        -1.22177481],
       [ 1.14424882,  0.55903879, -0.0442804 ,  0.0961888 ,  0.31122317,
        -2.06641918]])

# Let's move to final step before we get the next token

after the data pipes pass through transformer block n block (currently n = 1 lol) we got

In [49]:
print(dataPipes1)

[[ 1.22173199  0.06118935  0.92162592 -0.234181   -0.07463095 -1.89573531]
 [ 0.37908953  0.12216531  1.88475371 -0.49791942 -0.58439609 -1.30369304]
 [-0.53468284  2.06320487 -0.23459028  0.16496192 -0.33519962 -1.12369404]
 [ 0.11085028  1.69449724  0.80869962 -0.68130134 -0.710971   -1.22177481]
 [ 1.14424882  0.55903879 -0.0442804   0.0961888   0.31122317 -2.06641918]]


In [50]:
dataPipes1.shape

(5, 6)

ในการทำนาย Token ถัดไป 

เราจะใช้ hidden state ของตำแหน่งสุดท้ายเท่านั้น

In [51]:
last_hidden = dataPipes1[-1]
print(last_hidden)
print('shape of lass hidden state : ', last_hidden.shape)

[ 1.14424882  0.55903879 -0.0442804   0.0961888   0.31122317 -2.06641918]
shape of lass hidden state :  (6,)


ในเชิงความหมายในขั้นตอนนี้

เวคเตอร์นี้คือ "โมเดลเข้าใจประโยคทั้งหมดแล้ว และพร้อมจะเดาคำถัดไป"

Shape : 
$$last\ hidden \in R^{d_{model}}$$ หรือ $$R^{(d_{model}, )}$$

เป้าหมายขั้นสุดท้าย

สมมุติให้ vocab ทั้งหมดมี 20 ตัว
จะได้ output เป็น

$$P(token_1), P(token_2), ..., P(token_{19})$$

ดังนั้นเราต้องแปลง: array จากขนาด $6 \rightarrow 20$

ซึ่ง Transformer block ทำไม่ได้
เป็นหน้าที่ของ LM Head (Output Linear Layer)

## Last Linear layer (LM Head)

Mathemetic definition

$$Logits = h_{last} \cdot W_{vocab} + b_{vocab}$$

Where 
* $h_{last} \in R^{(d_{model}, )}$
* $W_{vocab} \in R^{(d_{model} \times vocab\_size)}$
* $b_{vocab} \in R^{(vocab\_size,)}$ 
* $logits \in R^{(vocab\_size,)}$ 

In [52]:
print("d_model =", d_model)

d_model = 6


In [53]:
vocab_size = 20

W_vocab = np.random.randn(d_model, vocab_size)
b_vocab = np.random.randn(vocab_size)

In [54]:
print(W_vocab.shape)

(6, 20)


In [55]:
logits = np.dot(last_hidden, W_vocab) + b_vocab

In [56]:
print(logits)

[ 0.46615844  2.8373876  -1.10474188  0.49014766 -2.94222228  2.09892103
 -1.5653536   3.53505459  0.94667025  1.82906255  0.79358703  0.69320932
  1.38484073  2.58051933 -6.43650825 -1.77985827  1.57963474  0.48230212
  0.78858574 -3.15814906]


if the logit result is   
[ 3.05492994  3.04876717  0.10503566 -0.08815435  2.14139093 -2.3520933
 -3.40995254  4.67951357 -3.81675782 -0.67168809  1.88311049  0.35795136
 -2.11323825 -1.29417076  1.21322951 -0.75312505 -0.32719262 -1.49954131
 -0.33330187 -0.06028637]

ความหมายคือ:
* token #8 (logit = 4.67) มีโอกาสมากกว่า token #1 (logit = 3.05)
* ค่ายังไม่ใช่ probability
* เป็นแค่คะแนนความเหมาะสม

### Softmax -> probability

Softmax ทำแค่ "normalize"

In [57]:
probs = softmax(logits)
probs.shape

(20,)

In [58]:
probs

array([1.55580983e-02, 1.66635777e-01, 3.23387458e-03, 1.59358377e-02,
       5.14891320e-04, 7.96262557e-02, 2.04024368e-03, 3.34781287e-01,
       2.51559163e-02, 6.07936535e-02, 2.15852430e-02, 1.95237598e-02,
       3.89883772e-02, 1.28887791e-01, 1.56374689e-05, 1.64635637e-03,
       4.73732438e-02, 1.58113016e-02, 2.14775584e-02, 4.14896491e-04])

### ความหมายเชิงผลลัพธ์

In [59]:
for i, p in enumerate(probs):
    print(f"token_{i}: {p:.4f}")

token_0: 0.0156
token_1: 0.1666
token_2: 0.0032
token_3: 0.0159
token_4: 0.0005
token_5: 0.0796
token_6: 0.0020
token_7: 0.3348
token_8: 0.0252
token_9: 0.0608
token_10: 0.0216
token_11: 0.0195
token_12: 0.0390
token_13: 0.1289
token_14: 0.0000
token_15: 0.0016
token_16: 0.0474
token_17: 0.0158
token_18: 0.0215
token_19: 0.0004


In [60]:
next_token_id = np.argmax(probs)
next_token_id

np.int64(7)

In [61]:
next_token_id = np.random.choice(
    range(vocab_size),
    p=probs
)

In [62]:
next_token_id

np.int64(13)